In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 10
Maxiterations_turbine = 10

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

#Время зарядки/разрядки, часы
time_ac = 4
#Время ожидания, часы
time_jdat = 12
#Конструкция аккумулятора
constr = {'Diametr':20, 'kolichestvo':8, 'Visota':20, 'lambda_min_vata':0.045, 'delta_min_vata':0.01}
PKM_zaryad = False
PKM_razryad = True
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 20
pkm_pgu_tol = 10**-2

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]


########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


########РАСЧЕТ расхода пара на ПКМ из условия заполнения хранилища################
########РАСЧЕТ мощности ГТУ из условия тепловой мощности и ################

################Расчет минимальной нагрузки ГТУ при остальных нормальных условиях############

Max_iterations_minimum = 10

if Сalculate_minimum == True:
    n_GTU = GTU_input.at["n", 1]
    start_time = time.time()
    for i in range(Max_iterations_minimum):
        if i < 6:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                Iter_pkm,
            ) = (2, 2, 15, 3)
        else:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                Iter_pkm,
            ) = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
                Iter_pkm,
            )

        Delt_Gcnd = 100
        Delt_Nturb = 100
        Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
        Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
        Delt_Gvd = (
            water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
            - 0.25
        ) / 0.25
        Delt_Gnd = (
            water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
            - 0.5
        ) / 0.5
        Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd)
        if n_GTU == 1 and Delta_min < 0:
            print("Мощность ГТУ 100% и расход пара все еще слишком мал")
        n_GTU = n_GTU - Delta_min / 8
        GTU_input.at["n", 1] = n_GTU

        arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
            New_iterations_KU_TU,
            New_iterations_cotel,
            New_iterations_turbine,
        )
        gas_streams = Calculate_CCGT_PKM_iter(arguments_all_it, Iter_pkm, pkm_pgu_tol)
        print(
            f"Время {i+1} итерации расчета мощности ГТУ при ПГУ с ПКМ: --- {round((time.time() - start_time), 1)} сек. ---"
        )
        print(f"Отклонение от ограничения минимальное равно {Delta_min}")
        print(f"n_GTU: {n_GTU}")
        if abs(Delta_min) < Calctolerance:
            gas_streams = Calculate_CCGT_PKM_iter(
                arguments_all_it, Iter_pkm, pkm_pgu_tol
            )
            print(f"Отклонение от ограничения минимальное равно {Delta_min}")
            print(f"Относительная мощность ГТУ равна {n_GTU}")
            print(
                f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
            )
            break
        if i == Max_iterations_minimum - 1:
            print(
                "Достигнуто максимальное количество итераций минимального расхода в ПГУ",
                i + 1,
            )

+Тепла в ГВТО хватает на теплофикацию
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  7.8 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 2 итерации расчета КУ+ТУ:---  14.0 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ: 2
Pnd_it [0.66412, 0.67582]
Pvd_it [8.13475, 8.23206]
fin КУ и ТУ:--- 14.0 сек. ---
Время 1 итерации расчета КУ+ТУ с ПКМ: --- 14.6 сек. ---
+Тепла в ГВТО хватает на теплофикацию
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  7.9 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 2 итерации расчета КУ+ТУ:---  13.1 сек. ---
Достигнуто максимальн

+Тепла в ГВТО хватает на теплофикацию
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  9.1 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 2 итерации расчета КУ+ТУ:---  17.2 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ: 2
Pnd_it [0.51357, 0.50747]
Pvd_it [6.05238, 5.9546]
fin КУ и ТУ:--- 17.2 сек. ---
Время 2 итерации расчета КУ+ТУ с ПКМ: --- 32.6 сек. ---
+Тепла в ГВТО хватает на теплофикацию
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  8.3 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достиг

KeyboardInterrupt: 

In [ ]:
gas_streams

In [2]:
water_streams

,T,P,H,G,S,X
AIR,0.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,471.967237,5.372089,3364.254056,39.269450,NaN,NaN
IVD-PEVD,270.982876,5.588086,2788.862748,39.269450,NaN,NaN
EVD-IVD,270.982876,5.588086,1256.608933,39.269450,NaN,NaN
PEN-EVD,164.625443,5.588086,698.393423,39.269450,NaN,NaN
BND-PEN,149.655730,0.471787,630.694097,39.269450,NaN,NaN
PPND-DROSND,194.905562,0.467608,2846.633013,10.803946,NaN,NaN
IND-PPND,149.655730,0.471787,2745.511279,10.803946,NaN,NaN
GPK-IND,149.655730,0.471787,634.268412,50.073396,NaN,NaN
GPK-REC,149.655730,0.471787,634.268412,60.674341,NaN,NaN


In [3]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COOL-HTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HTS-HTSCOOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HTSCOOL-Separ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separ-SGaccum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SGaccum-Separacc,5.703306,2.0,1075.096789,365775.995050,0.000000,0.000000,0.168738,0.0,0.050357,0.097508,0.681387,0.002009
Separacc-Sepout,5.703306,2.0,957.542925,336452.977654,0.000000,0.000000,0.177686,0.0,0.000000,0.102679,0.717520,0.002115
Sepout-COMB,5.703306,2.0,957.542925,22.283679,0.000000,0.000000,0.177686,0.0,0.000000,0.102679,0.717520,0.002115
Sepout-GTU,5.703306,2.0,957.542925,1.081111,0.000000,0.000000,0.177686,0.0,0.000000,0.102679,0.717520,0.002115
AIR-COMP,15.000000,0.1,293.187169,775.353423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
electric

In [ ]:
heaters

In [ ]:
accumulation